In [1]:
# Only use "ADNIMERGE.csv" as phenotype and covariate data.

import numpy as np
import pandas as pd

Load sample names of VCF data.

In [2]:
VCFsamplelist = pd.read_csv("ADNI/WGS_Omni2.5M_20140220/vcfsample_list.txt", header = None)
VCFsamplelist_IND = VCFsamplelist.applymap(lambda x: x.split("_")[0]).iloc[:,0]
VCFsamplelist_IND = VCFsamplelist_IND.rename('IND')
VCFsamplelist_ID = VCFsamplelist.applymap(lambda x: x.split("_")[1]).iloc[:,0]
VCFsamplelist_ID = VCFsamplelist_ID.rename('ID')
VCFsamplelist = pd.concat([VCFsamplelist_IND,VCFsamplelist_ID],axis=1)

In [6]:
VCFsamplelist = VCFsamplelist.rename(columns={'IND':'FID', 'ID':'IID'})
VCFsamplelist = VCFsamplelist.sort_values('IID')
VCFsamplelist

,FID,IID
16,17,0002
581,587,0008
316,319,0015
616,622,0021
688,694,0023
...,...,...
668,674,4676
732,739,4686
483,488,4692
749,756,4696


In [10]:
# Merge summary data.
ADNI_BASIC_data = pd.read_csv("ADNI/Cov/ADNIMERGE.csv")
ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data.iloc[:,4]!="ADNI3"]
ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data["VISCODE"]=="bl"]
ADNI_BASIC_data = ADNI_BASIC_data[~np.isnan(ADNI_BASIC_data["AV45_bl"])]
ADNI_BASIC_data = ADNI_BASIC_data[["RID", "VISCODE", "AGE", "PTGENDER", "APOE4", "AV45_bl", "EXAMDATE"]]

ADNI_BASIC_data = ADNI_BASIC_data.astype({"RID": "int64"})
ADNI_BASIC_data["RID"] = ADNI_BASIC_data["RID"].apply(lambda x:str(x).zfill(4))
ADNI_BASIC_data = ADNI_BASIC_data.sort_values("RID")

ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data["RID"].isin(VCFsamplelist["IID"])]

In [11]:
def gender2val(df):
    if df["PTGENDER"] == "Male":
        return 1
    elif df["PTGENDER"] == "Female":
        return 0

for rid in ADNI_BASIC_data["RID"].iteritems():
    ridval = rid[1]
    ADNI_BASIC_data.loc[ADNI_BASIC_data["RID"]==ridval,"FID"] = VCFsamplelist.loc[VCFsamplelist["IID"]==ridval,"FID"].values
ADNI_BASIC_data["SEX"] = ADNI_BASIC_data.apply(gender2val, axis=1)
ADNI_BASIC_data = ADNI_BASIC_data.rename(columns={'RID': 'IID'})
ADNI_BASIC_data = ADNI_BASIC_data[["FID","IID","AV45_bl","AGE","SEX","APOE4","EXAMDATE"]]
ADNI_BASIC_data = ADNI_BASIC_data.astype({"FID": "int64"})
ADNI_BASIC_data = ADNI_BASIC_data.sort_values("FID")


ADNI_BASIC_data_cov = ADNI_BASIC_data[["FID","IID","AGE","SEX","APOE4"]]
ADNI_BASIC_data_cov.to_csv("cov_merge_ADNI_SUVR.txt", sep="\t", index=False)

ADNI_BASIC_data_pheno = ADNI_BASIC_data[["FID","IID","AV45_bl"]]
ADNI_BASIC_data_pheno.to_csv("pheno_merge_ADNI_SUVR.txt", sep="\t", index=False)

In [12]:
ADNI_BASIC_data

,FID,IID,AV45_bl,AGE,SEX,APOE4,EXAMDATE
12148,5,2237,0.899074,65.0,1,0.0,2011-01-26
3042,10,2264,1.308140,69.3,1,1.0,2011-03-11
3008,11,2216,1.331980,68.1,1,1.0,2011-03-10
4916,23,2164,1.170410,70.9,0,0.0,2010-12-02
3002,25,2205,1.293240,84.7,1,0.0,2010-12-09
...,...,...,...,...,...,...,...
8263,814,4259,1.267950,63.0,0,1.0,2011-11-07
7802,815,4302,1.225900,76.1,1,0.0,2011-11-18
2928,816,2109,1.427520,75.2,1,0.0,2010-10-18
8336,817,4244,0.868518,91.4,1,0.0,2011-11-22
